# 🤖 Build a Text2SQL ReAct Agentic AI System with LangChain

## Learning Objectives
In this notebook, you will learn:
1. **ReAct Pattern** - Reasoning + Acting framework for AI agents
2. **SQL Database Toolkit** - LangChain's tools for database interaction
3. **LangGraph Agents** - Modern agent implementation with `create_react_agent`
4. **Agent vs Workflow** - When to use autonomous agents

## Agent Architecture
Unlike the deterministic workflow approach, this **ReAct Agent** is fully autonomous:

```
User Question → [Agent: Think → Act → Observe → Repeat] → Final Answer
```

The agent uses a **ReAct (Reasoning + Acting) loop**:
1. **THINK**: Analyze the question and decide what to do
2. **ACT**: Use a tool (list tables, get schema, execute query, etc.)
3. **OBSERVE**: Analyze the tool's output
4. **REPEAT**: Continue until the answer is complete

![ReAct Agent Architecture](https://i.imgur.com/X9ZKMZM.png)

## Prerequisites
- Understanding of the Text2SQL Workflow notebook
- Familiarity with agent concepts
- OpenAI API key

---
## 🔄 How This Differs from the Workflow Approach

Both notebooks solve the same problem (**Natural Language → SQL → Answer**), but with **fundamentally different architectures**:

### Comparison Table

| Aspect | 📊 Workflow (Notebook 1) | 🤖 ReAct Agent (This Notebook) |
|--------|-------------------------|--------------------------------|
| **Execution Path** | Fixed, deterministic chain | Autonomous, dynamic decisions |
| **Schema Discovery** | Pre-configured in prompt | Agent explores schema on-the-fly |
| **Error Handling** | Fails if SQL is wrong | Self-corrects and retries |
| **Flexibility** | Fixed: Generate → Execute → Answer | Agent decides: List → Schema → Validate → Execute |
| **Control Level** | High (you define every step) | Lower (agent decides steps) |
| **Token Usage** | Lower (single LLM call per step) | Higher (multiple reasoning steps) |
| **Debugging** | Easy (predictable path) | Harder (dynamic path) |
| **Best For** | Simple, predictable queries | Complex, exploratory queries |

### Visual Comparison

**Workflow Approach (Notebook 1):**
```
Question → [Generate SQL] → [Execute SQL] → [Format Answer] → Response
         (Fixed chain, no decisions)
```

**ReAct Agent Approach (This Notebook):**
```
Question → [Think: What tables exist?]
         → [Act: list_tables tool]
         → [Observe: Branch, Comic, Customer...]
         → [Think: Need Customer and Sale schema]
         → [Act: get_schema tool]
         → [Observe: CREATE TABLE...]
         → [Think: Now I can write the query]
         → [Act: validate_query tool]
         → [Act: execute_query tool]
         → Response
```

### When to Use Each:

| Use Case | Recommended Approach |
|----------|---------------------|
| Simple, predictable queries | 📊 Workflow |
| Cost-sensitive applications | 📊 Workflow |
| Need strict control/auditability | 📊 Workflow |
| Complex multi-table queries | 🤖 ReAct Agent |
| Unknown/changing database schema | 🤖 ReAct Agent |
| Need self-correction on errors | 🤖 ReAct Agent |
| Interactive exploration | 🤖 ReAct Agent |

---
## 📦 Step 1: Environment Setup

Let's set up our environment with necessary dependencies including **LangGraph** (required for ReAct agents).

In [ ]:
# ============================================================================
# ENVIRONMENT SETUP: Suppress Warnings & Install Dependencies
# ============================================================================
# KEY DIFFERENCE: This notebook requires LangGraph!
# LangGraph is the modern way to build agents in LangChain.
# It provides create_react_agent() - a robust ReAct agent implementation.
# ============================================================================

from warnings import filterwarnings
filterwarnings('ignore')

# Uncomment to install packages if needed:
# !pip install langchain==0.3.14
# !pip install langchain-openai==0.3.0
# !pip install langchain-community==0.3.14
# !pip install langgraph==0.2.62  # ← NEW! Required for ReAct agents

print("✅ Environment setup complete!")

In [ ]:
# ============================================================================
# LOAD API KEYS FROM ENVIRONMENT
# ============================================================================
# Best practice: Store API keys in a .env file, never in code!
#
# Your .env file should contain:
#   OPENAI_API_KEY=your_api_key_here
# ============================================================================

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

print("✅ Environment variables loaded successfully!")

---
## 🗄️ Step 2: Database Setup

We'll use the same **Comic Store Database** as the workflow notebook.

In [ ]:
# ============================================================================
# DATABASE SETUP: Create SQLite Database
# ============================================================================
# Same database as the workflow notebook.
# Tables: Branch, Publisher, Comic, Customer, Employee, Inventory, Sale, SaleTransactions
# ============================================================================

# Verify SQLite and create database
!sqlite3 --version
!sqlite3 ComicStore.db ".read ./comicdb_create_script.sql"

print("\n✅ Database created successfully!")

In [ ]:
# ============================================================================
# LANGCHAIN SQL DATABASE CONNECTION
# ============================================================================
# SQLDatabase wraps the database - same as workflow notebook.
# But here, the AGENT will use this to explore the schema dynamically!
# ============================================================================

from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///ComicStore.db")

print(f"🔗 Database Dialect: {db.dialect}")
print(f"📋 Available Tables: {db.get_usable_table_names()}")

---
## 🔧 Step 3: Create SQL Database Tools

### 🔄 KEY DIFFERENCE FROM WORKFLOW!

In the **Workflow approach**, we explicitly defined:
- A query generation chain
- A query execution chain
- An answer generation chain

In the **ReAct Agent approach**, we give the agent **tools** and let it decide how to use them:

| Tool | What Agent Can Do With It |
|------|---------------------------|
| `sql_db_list_tables` | Discover what tables exist |
| `sql_db_schema` | Understand table structure |
| `sql_db_query_checker` | Validate SQL before running |
| `sql_db_query` | Execute SQL and get results |

In [ ]:
# ============================================================================
# CREATE SQL DATABASE TOOLKIT
# ============================================================================
# KEY DIFFERENCE: Instead of building fixed chains, we create TOOLS!
#
# SQLDatabaseToolkit provides all the tools an agent needs:
# 1. sql_db_list_tables: "What tables exist?" → Returns table names
# 2. sql_db_schema: "What's in this table?" → Returns CREATE TABLE + samples
# 3. sql_db_query_checker: "Is this SQL valid?" → Validates syntax via LLM
# 4. sql_db_query: "Run this query" → Executes and returns results
#
# The agent decides WHICH tool to use and WHEN, based on its reasoning!
# ============================================================================

from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI

# Initialize the LLM for tools (temperature=0 for consistent SQL)
chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)

# Create the toolkit with all SQL tools
sql_toolkit = SQLDatabaseToolkit(db=db, llm=chatgpt)

# Extract the tools list
sql_tools = sql_toolkit.get_tools()

print("✅ SQL Database Toolkit created!")
print(f"\n🔧 Available Tools ({len(sql_tools)}):")
print("=" * 60)
for tool in sql_tools:
    print(f"  • {tool.name}: {tool.description[:80]}...")

---
## 📝 Step 4: Define the System Prompt

The system prompt is **critical** for agent behavior. It defines:
- The agent's role and purpose
- Rules for SQL generation
- Safety constraints (no DML operations)
- Step-by-step workflow guidance
- Output formatting

In [ ]:
# ============================================================================
# DEFINE THE SYSTEM PROMPT FOR THE ReAct AGENT
# ============================================================================
# KEY DIFFERENCE: This prompt guides AUTONOMOUS behavior!
#
# In the workflow, the prompt just said "generate SQL". Here, we:
# 1. Tell the agent to ALWAYS list tables first (exploration)
# 2. Tell it to get schema before writing queries
# 3. Tell it to validate queries before execution
# 4. Add safety constraints (no DML operations)
#
# Using raw string (r"...") to avoid SyntaxWarning with \$ in the string.
# ============================================================================

from langchain_core.messages import SystemMessage

SQL_PREFIX = r"""You are an agent designed to interact with a SQL database in SQLite.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
Make sure you generate a correct SQLite query as plain text without any formatting or code blocks.
Do not include sql or similar markers.
Do not try to explain the query, just provide the query as-is, like this: SELECT ...

You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.

You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database even if the user asks.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables.

When generating the final answer in markdown from the results,
if there are special characters in the text, such as the dollar symbol,
ensure they are escaped properly for correct rendering e.g $25.5 should become \$25.5
"""

# Wrap the prompt in a SystemMessage for use with the agent
SYS_PROMPT = SystemMessage(content=SQL_PREFIX)

print("✅ System prompt configured successfully!")
print(f"📋 Prompt length: {len(SQL_PREFIX)} characters")

---
## 🤖 Step 5: Create the ReAct Agent

### 🔄 KEY DIFFERENCE FROM WORKFLOW!

**Workflow (Notebook 1):**
```python
# Fixed chain - always does the same steps
chain = query_write_chain | query_execute_chain | answer_chain
```

**ReAct Agent (This Notebook):**
```python
# Autonomous agent - decides what to do at each step
agent = create_react_agent(model, tools, system_prompt)
```

In [ ]:
# ============================================================================
# CREATE THE ReAct AGENT USING LANGGRAPH
# ============================================================================
# KEY DIFFERENCE: create_react_agent builds an AUTONOMOUS agent!
#
# Unlike chains that have a fixed execution path, this agent:
# 1. THINKS about what to do (Reasoning)
# 2. Takes an ACTION using tools (Acting)
# 3. OBSERVES the result
# 4. DECIDES if it needs more information or can answer
# 5. REPEATS until it has the final answer
#
# Parameters:
# - model: The LLM brain (GPT-4o)
# - tools: What the agent can use (our SQL tools)
# - state_modifier: System prompt guiding behavior
# ============================================================================

from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

# Create the ReAct agent with our SQL tools and system prompt
text2sql_agent = create_react_agent(
    model=chatgpt,            # The LLM brain
    tools=sql_tools,          # Tools for database interaction
    state_modifier=SYS_PROMPT # System instructions
)

print("✅ Text2SQL ReAct Agent created successfully!")
print("\n📊 Agent Graph Visualization:")
text2sql_agent  # Display the agent graph

---
## 🧪 Step 6: Test the ReAct Agent

Watch the agent's **ReAct loop** in action! You'll see it:
1. List tables (exploration)
2. Get schemas (understanding)
3. Validate SQL (safety check)
4. Execute query (action)
5. Format answer (response)

In [ ]:
# ============================================================================
# TEST THE AGENT - OBSERVE THE REACT LOOP
# ============================================================================
# KEY DIFFERENCE: Watch the agent THINK and DECIDE!
#
# Unlike the workflow which just generates SQL → executes → answers,
# the agent will:
# 1. Think: "I need to know what tables exist"
# 2. Act: Call sql_db_list_tables
# 3. Observe: "Tables are: Comic, Sale, Customer..."
# 4. Think: "I need the schema for Sale and Comic"
# 5. Act: Call sql_db_schema
# ... and so on until it has the answer!
# ============================================================================

from IPython.display import display, Markdown

print("🤖 Agent Query: 'What are the Top 10 most popular comics?'")
print("=" * 70)
print("\n📋 OBSERVE THE REACT LOOP IN ACTION:\n")

for event in text2sql_agent.stream(
    {"messages": [HumanMessage(content="What are the Top 10 most popular comics?")]},
    stream_mode='values'  # Returns full agent state with all messages
):
    # pretty_print() formats the output nicely
    event["messages"][-1].pretty_print()

print('\n' + '=' * 70)
print('📋 FINAL RESPONSE:')
print('=' * 70)
display(Markdown(event["messages"][-1].content))

In [ ]:
# ============================================================================
# HELPER FUNCTION FOR CLEANER AGENT INTERACTION
# ============================================================================
# This function wraps the agent interaction for cleaner code.
# In production applications, you typically hide internal workings
# and just show the final result.
#
# Parameters:
# - query: The natural language question to ask
# - verbose: If True, shows the full reasoning chain; if False, only final answer
# ============================================================================

def call_text2sql_agent(query: str, verbose: bool = False) -> None:
    """
    Query the Text2SQL agent with a natural language question.
    
    Args:
        query: The question to ask (e.g., "What are the top 5 customers?")
        verbose: If True, prints the full reasoning chain; if False, only final answer
    """
    print(f"🤖 Query: '{query}'")
    print("=" * 70)
    
    for event in text2sql_agent.stream(
        {"messages": [HumanMessage(content=query)]},
        stream_mode='values'
    ):
        if verbose:
            event["messages"][-1].pretty_print()

    print('\n📋 Final Response:\n')
    display(Markdown(event["messages"][-1].content))

print("✅ Helper function 'call_text2sql_agent' defined successfully!")

In [ ]:
# ============================================================================
# EXAMPLE: Customer Analytics Query
# ============================================================================
# The agent will automatically:
# 1. Identify relevant tables (Customer, Sale, SaleTransactions)
# 2. Join tables as needed
# 3. Aggregate and sort results
# ============================================================================

call_text2sql_agent(
    query="What are the top 5 customers with most comics purchased?",
    verbose=False  # Set to True to see the agent's step-by-step reasoning
)

In [ ]:
# ============================================================================
# EXAMPLE: Revenue Analysis Query
# ============================================================================
# This query requires joining Customer with Sale and aggregating amounts.
# ============================================================================

call_text2sql_agent(
    query="Which are the top 5 customers with most money spent?",
    verbose=False
)

In [ ]:
# ============================================================================
# EXAMPLE: Employee Performance Query
# ============================================================================
# The agent understands "salesman" refers to the Employee table
# and joins with Sale to calculate revenue.
# ============================================================================

call_text2sql_agent(
    query="Which are the top 3 salesmen with highest revenue?",
    verbose=False
)

---
## 🎯 Summary and Key Takeaways

### What We Built
We created a **Text2SQL ReAct Agentic AI System** that can:
1. ✅ Understand natural language questions about our database
2. ✅ Automatically explore the database schema
3. ✅ Generate and validate SQL queries
4. ✅ Self-correct on errors
5. ✅ Execute queries and return human-readable results

### Key Differences from Workflow (Notebook 1)

| Feature | Workflow | ReAct Agent |
|---------|----------|-------------|
| **Architecture** | Fixed chain of components | LangGraph graph with tool nodes |
| **Execution** | Always same steps | Dynamically decides steps |
| **Schema Knowledge** | Must be in prompt | Agent discovers it |
| **Error Recovery** | Fails on errors | Retries with fixes |
| **Dependencies** | LangChain only | LangChain + LangGraph |

### Key Components in This Notebook

| Component | Purpose |
|-----------|---------|  
| **SQLDatabase** | LangChain wrapper for database interaction |
| **SQLDatabaseToolkit** | Provides SQL tools (query, schema, list, checker) |
| **System Prompt** | Guides agent behavior and ensures safety |
| **create_react_agent** | LangGraph's modern ReAct agent implementation |

### Best Practices Demonstrated
- ✅ Use raw strings for prompts with special characters
- ✅ Set `temperature=0` for consistent SQL generation
- ✅ Include safety constraints in system prompt (no DML)
- ✅ Always validate queries before execution
- ✅ Use LangGraph's `create_react_agent` (modern approach)

In [ ]:
# ============================================================================
# 🧪 TRY YOUR OWN QUERIES!
# ============================================================================
# Modify the query below to ask your own questions about the database.
# 
# Example questions you can try:
# - "What is the average price of comics by genre?"
# - "Which branch has the most inventory?"
# - "List all comics published before 2000"
# - "Who are the customers from New York?"
# - "What are the total sales for each month?"
# ============================================================================

# Uncomment and modify to test your own questions:

# call_text2sql_agent(
#     query="Your question here...",
#     verbose=True  # Set to True to see the agent's reasoning
# )